In [3]:
import sounddevice as sd
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import AutoProcessor, AutoModelForSeq2SeqLM
from datasets import load_dataset
from transformers import AutoProcessor, AutoModel
from IPython.display import Audio

In [5]:
whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")

suno_processor = AutoProcessor.from_pretrained("suno/bark-small")
suno_model = AutoModel.from_pretrained("suno/bark-small")

In [6]:
# Set up the TTS pipeline
tts_pipeline = pipeline("text-to-speech")

# Define the sample rate and duration for recording
sample_rate = 16000  #
duration = 5  #secs

while True:
    # Record audio from the microphone
    print("Recording...")
    audio_data = sd.rec(int(sample_rate * duration), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()  
    print("Recording complete")

    # Convert to an array
    audio_array = np.array(audio_data).squeeze()

    # Preprocess the audio data and generate token IDs
    input_features = whisper_processor(audio_array, sampling_rate=sample_rate, return_tensors="pt").input_features
    predicted_ids = whisper_model.generate(input_features)

    # Decode the token IDs to get the transcription
    transcription = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)
    print("Transcription: ", transcription[0])

    voice_preset = "v2/en_speaker_6"
    inputs = processor(
        text=[transcription[0]],
        return_tensors="pt",
        voice_preset = voice_preset
    )
    speech_values = model.generate(**inputs)

    # Decode the output tokens to get the modified text
    output_text = suno_processor.decode(outputs[0], skip_special_tokens=True)
    print("Output text: ", output_text)

    # Convert the output text to speech using the TTS pipeline
    speech_output = tts_pipeline(output_text)

    # Save the speech audio to a file
    with open("output.wav", "wb") as f:
        f.write(speech_output["audio"])

    # Play the speech audio through the speakers
    wave_obj = sa.WaveObject.from_wave_file("output.wav")
    play_obj = wave_obj.play()
    play_obj.wait_done()

NameError: name 'pipeline' is not defined

In [19]:
# record audio
sample_rate = 16000  #hz
duration = 5  # in secs
print("Recording...")
audio_data = sd.rec(int(sample_rate * duration), samplerate=sample_rate, channels=1, dtype='int16')
sd.wait()  
print("Recording complete")

Recording...
Recording complete


In [21]:
# convert to array
audio_array = np.array(audio_data).squeeze()

# Preprocess the audio data and generate token IDs
input_features = processor(audio_array, sampling_rate=sample_rate, return_tensors="pt").input_features
predicted_ids = model.generate(input_features)

# Decode the token ids to get the transcription
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print("Transcription: ", transcription[0])

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [8]:
# Decode the token ids to get the transcription
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print("Transcription: ", transcription[0])

Transcription:   Hello, hello, hello, hello, my name's Connor. Hello, my name's Connor. Hello, my name's Con-


In [26]:
voice_preset = "v2/en_speaker_6"
inputs = processor(
    text=[transcription[0]],
    return_tensors="pt",
    voice_preset = voice_preset
)
speech_values = model.generate(**inputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [27]:
from IPython.display import Audio
sampling_rate = model.generation_config.sample_rate
Audio(speech_values.cpu().numpy().squeeze(), rate=sampling_rate)